In [1]:
import os
import pyodbc
import pandas as pd
from arcgis.gis import GIS

In [4]:
#loading the estimates data 
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=DDAMWSQL16.sandag.org;'
                      'Database=estimates;'
                      'Trusted_Connection=yes;')

hhpop_2020 ="SELECT [mgra_id], [yr_id], [housing_type_id], [population]"\
                  "FROM [estimates].[est_2020_01].[dw_population]"\
                   "WHERE housing_type_id= 1"
                  
             
hhpop = pd.read_sql_query(hhpop_2020, conn)    

print(hhpop.head())

uniqueyr = hhpop['yr_id'].nunique()
print(uniqueyr)

      mgra_id  yr_id  housing_type_id  population
0  1400000101   2010                1          41
1  1400000102   2010                1           0
2  1400000201   2010                1          74
3  1400000202   2010                1           0
4  1400000301   2010                1         126
11


In [6]:
# loading the dim_mgra_denormalized table from demographic warehouse 
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=DDAMWSQL16.sandag.org;'
                      'Database=demographic_warehouse;'
                      'Trusted_Connection=yes;')

dim_mgra ="SELECT *"\
                  "FROM [demographic_warehouse].[dim].[mgra_denormalize]"\
                   "WHERE series= 14 " 
             
dim_mgra = pd.read_sql_query(dim_mgra, conn)    

dim_mgra.dtypes


mgra_id             int64
mgra                int64
series              int64
region             object
sra                object
tract              object
supervisorial      object
college            object
cpa                object
jurisdiction       object
zip                object
secondary          object
elementary         object
msa                object
sdcouncil          object
transit            object
unified            object
jurisdiction_id     int64
cpa_id              int64
taz                 int64
dtype: object

In [8]:
# setting up the query string for loading the mgra data from geodepot

mgra_query = """SELECT  [OBJECTID]
      ,[MGRA]
      ,[CT10]
      ,[CT10BG]
      ,[SRA]
      ,[MSA]
      ,[City]
      ,[ZIP]
      ,[Sphere]
      ,[CPA]
      ,[CPASG]
      ,[TAZ]
      ,[Council]
      ,[Super]
      ,[LUZ]
      ,[Elem]
      ,[Unif]
      ,[High]
      ,[Coll]
      ,[Transit]
      ,[Shape] """ + \
    ".STGeometryN(1).ToString()"  + "FROM [GeoDepot].[gis].[MGRA13PT]"

 # + geo_col + '.STGeometryN(1).ToString()'  
 #  ,[Shape]
   #-  ,[GDB_GEOMATTR_DATA]

In [9]:
# loading the mgra data from geodepot

conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=sql2014b8;'
                      'Database=GeoDepot;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
             
mgra = pd.read_sql_query(mgra_query, conn)    

mgra.dtypes


OBJECTID     int64
MGRA         int64
CT10         int64
CT10BG       int64
SRA          int64
MSA          int64
City         int64
ZIP          int64
Sphere       int64
CPA          int64
CPASG        int64
TAZ          int64
Council      int64
Super        int64
LUZ          int64
Elem         int64
Unif         int64
High         int64
Coll         int64
Transit      int64
            object
dtype: object

In [10]:
# creating cross-walking between the age estimates table, dim_mgra table, and mgra geodepot file

mgra_shape= mgra.merge(dim_mgra, left_on='MGRA', right_on='mgra')
mgra_shape.shape[0]
mgra_shape.head()
estimates_mgra= mgra_shape.merge(hhpop, left_on= 'mgra_id', right_on= 'mgra_id')
print(estimates_mgra.shape[0])
estimates_mgra

# checking the number of rows in the final df remains intact during merge

print(estimates_mgra.shape[0]== uniqueyr*27316)


estimates_mgra.rename({"mgra": "MGRA"}, axis=1)

estimates_mgra.columns


300476
True


Index(['OBJECTID', 'MGRA', 'CT10', 'CT10BG', 'SRA', 'MSA', 'City', 'ZIP',
       'Sphere', 'CPA', 'CPASG', 'TAZ', 'Council', 'Super', 'LUZ', 'Elem',
       'Unif', 'High', 'Coll', 'Transit', '', 'mgra_id', 'mgra', 'series',
       'region', 'sra', 'tract', 'supervisorial', 'college', 'cpa',
       'jurisdiction', 'zip', 'secondary', 'elementary', 'msa', 'sdcouncil',
       'transit', 'unified', 'jurisdiction_id', 'cpa_id', 'taz', 'yr_id',
       'housing_type_id', 'population'],
      dtype='object')

In [18]:
# Loading GIS module from arcgis for visualizing estimates 

username = ""
password = ""
gis = GIS("https://www.arcgis.com", username, password)



Exception: Unable to generate token.
Too many invalid logins. Please try again later.
(Error Code: 400)

In [12]:
mgra13 = gis.content.search("MGRA", item_type="Feature Layer", )[4]
mgra_map = gis.map("San Diego", zoomlevel=16)

In [13]:
mgra_map.add_layer(mgra13, {"field_name":"MGRA" })

In [14]:
mgra_map

MapView(layout=Layout(height='400px', width='100%'))

In [15]:
# viewing the attribute table for the mgra13 layer 
mgra_layer= mgra13.layers[0]

sdf = pd.DataFrame.spatial.from_layer(mgra_layer)
sdf.head()

,CPA,CPASG,CT10,CT10BG,City,Coll,Council,Elem,High,LUZ,...,MSA,OBJECTID,SHAPE,SRA,Sphere,Super,TAZ,Transit,Unif,ZIP
0,1442,0,100,1001,14,4,3,0,0,95,...,0,1,"{""rings"": [[[-13045374.349833563, 3863379.4859...",1,1442,4,3331,1,30,92103
1,1442,0,100,1001,14,4,3,0,0,95,...,0,2,"{""rings"": [[[-13044868.53026565, 3863126.96744...",1,1442,4,3331,1,30,92103
2,1442,0,100,1001,14,4,3,0,0,95,...,0,3,"{""rings"": [[[-13044573.865921693, 3863107.2344...",1,1442,4,3358,1,30,92103
3,1442,0,100,1001,14,4,3,0,0,95,...,0,4,"{""rings"": [[[-13044254.604501536, 3862633.8466...",1,1442,4,3358,1,30,92103
4,1442,0,100,1001,14,4,3,0,0,95,...,0,5,"{""rings"": [[[-13044782.283246167, 3862848.1879...",1,1442,4,3358,1,30,92103


In [53]:
estimates_mgra= sdf.merge(estimates_mgra, left_on='MGRA', right_on='MGRA')

estimates_mgra.tail()

,CPA,CPASG,CT10,CT10BG,City,Coll,Council,Elem,High,LUZ,...,msa,sdcouncil,transit,unified,jurisdiction_id,cpa_id,taz,yr_id,housing_type_id,population
300471,0,0,20405,204052,7,3,0,12,13,175,...,North County East,None,NCTD,None,19,1909,1378,2016,1,0
300472,0,0,20405,204052,7,3,0,12,13,175,...,North County East,None,NCTD,None,19,1909,1378,2017,1,0
300473,0,0,20405,204052,7,3,0,12,13,175,...,North County East,None,NCTD,None,19,1909,1378,2018,1,0
300474,0,0,20405,204052,7,3,0,12,13,175,...,North County East,None,NCTD,None,19,1909,1378,2019,1,0
300475,0,0,20405,204052,7,3,0,12,13,175,...,North County East,None,NCTD,None,19,1909,1378,2020,1,0


In [58]:
estimates_mgra[estimates_mgra.yr_id==2016].spatial.plot(kind='map', map_widget= mgra_map, 
                           renderer_type='u', # specify the unique value renderer using its notation 'u'
        col='MGRA')

KeyError: None